# Classification de tweets avec un LLM (Ollama + Mistral)

Dans ce notebook, on va :

1. Charger un fichier CSV de tweets nettoyés.
2. Préparer un prompt avec quelques exemples (few-shot).
3. Appeler un modèle LLM local (Ollama, modèle *mistral*) pour classer chaque tweet.
4. Récupérer la réponse du modèle en JSON et la parser proprement.
5. Normaliser les résultats (topics, sentiment, incident, etc.).
6. Fusionner les prédictions avec le CSV d’origine et sauvegarder le résultat.

Ce notebook reprend la logique du script Python, mais de manière plus pédagogique et étape par étape.


In [3]:
# Imports des bibliothèques nécessaires

import requests          # pour appeler l'API HTTP d'Ollama
import pandas as pd      # pour lire/écrire les CSV
import orjson           # pour parser le JSON renvoyé par le modèle
import time             # pour faire des pauses entre les retries
import sys              # pour quitter proprement (utile si fichier manquant)
from pathlib import Path # pour travailler proprement avec les chemins de fichiers
import os

# ====== CONFIG GENERALE ======

MODEL = "mistral"                                 # Nom du modèle Ollama

TEXT_COL = "text_clean_llm"                       # Nom de la colonne texte dans le CSV

# Dossier du projet = parent du dossier où est le notebook
PROJECT_ROOT = Path().resolve().parent
DATA_DIR = PROJECT_ROOT / "data"

#SOURCE_CSV = DATA_DIR / "free_tweet_export.cleaned.llm.csv"     #Fichier des tweets nettoyés
#OUTPUT_CSV = DATA_DIR / "free_tweet_classified_clean.csv"       #Fichier de sortie avec les prédictions

SOURCE_CSV = DATA_DIR / "echantillion_5_tweets.csv"              # échantillion de 5 tweets pour tester le code(prend moin du temps)
OUTPUT_CSV = DATA_DIR / "resultat_classification_echantillion_5_tweets.csv" # Fichier de test de sortie avec les prédictions

print("Chemin complet CSV :", SOURCE_CSV)

if not SOURCE_CSV.exists():
    print(f"Fichier introuvable: {SOURCE_CSV}")
else:
    df_full = pd.read_csv(SOURCE_CSV, engine="python")
    print(df_full.head())


# Paramètres pour éventuellement limiter le nombre de lignes (pour debug) :
SAMPLE_N = None        # par ex: 200 pour tester uniquement sur 200 lignes
SAVE_SAMPLE_AS = None  # ex: "echantillon.csv" pour sauvegarder l'échantillon

# Paramètres d'appel à l'API
API_URL = "http://localhost:11434/api/chat"
BATCH_SIZE = 20        # nombre de tweets à traiter par batch
HTTP_TIMEOUT = 300     # timeout HTTP en secondes
MAX_RETRIES = 3        # nombre de tentatives en cas d'erreur
BACKOFFS = [1, 3, 7]   # temps d'attente entre tentatives

# Session HTTP réutilisée pour optimiser les appels réseau
SESSION = requests.Session()


Chemin complet CSV : C:\Users\HP\Desktop\projet\data\echantillion_5_tweets.csv
                    id                  created_at  \
0  1343458257915031553  2020-12-28 08:26:23 +01:00   
1  1480465391424053249  2022-01-10 10:03:49 +01:00   
2  1485331097844129798  2022-01-23 20:18:24 +01:00   
3  1694675653205860558  2023-08-24 13:38:55 +02:00   
4  1695912123824546156  2023-08-27 23:32:13 +02:00   

                                           full_text  \
0  💩 à @free parce-que Débit Très instable, … \n\...   
1  🎂 @free célèbre aujourd'hui 10 ans de révoluti...   
2  Et vous ça se passe comment la fibre ?\n@Sosh_...   
3  Dans la lutte contre les installations sauvage...   
4  Hello @free, @Freebox et @orange je crois que ...   

                                               media   screen_name  \
0                                                 []      m_annuel   
1  [{"type":"video","url":"https://t.co/c1e4svvex...   GroupeIliad   
2  [{"type":"photo","url":"https://t.co/I41UXroh0

## Chargement des données

Dans cette partie, on va :

- Vérifier que le fichier CSV source existe.
- Charger le CSV dans un DataFrame `df_full`.
- Vérifier que la colonne texte choisie (`TEXT_COL`) est bien présente.
- (Optionnel) Prendre seulement un échantillon des données si `SAMPLE_N` est défini.


In [4]:
# Vérification de l'existence du fichier
if not Path(SOURCE_CSV).exists():
    print(f"Fichier introuvable: {SOURCE_CSV}")
    sys.exit(1)

# Lecture du fichier CSV complet
df_full = pd.read_csv(SOURCE_CSV, engine="python")
print(f"Colonnes trouvées dans le CSV : {list(df_full.columns)}")

# Vérification de la présence de la colonne texte
if TEXT_COL not in df_full.columns:
    raise SystemExit(f"Colonne {TEXT_COL} introuvable dans {SOURCE_CSV}")

# Gestion de l'échantillonnage
if SAMPLE_N is not None:
    df = df_full.head(int(SAMPLE_N)).copy()
    if SAVE_SAMPLE_AS:
        df.to_csv(SAVE_SAMPLE_AS, index=False, encoding="utf-8")
        print(f"💾 Échantillon sauvegardé dans {SAVE_SAMPLE_AS} ({len(df)} lignes)")
    else:
        print(f"🔎 Échantillon en mémoire: {len(df)} lignes (non sauvegardé)")
else:
    df = df_full.copy()
    print(f"🔎 Traitement de tout le fichier: {len(df)} lignes")

df.head()


Colonnes trouvées dans le CSV : ['id', 'created_at', 'full_text', 'media', 'screen_name', 'name', 'profile_image_url', 'user_id', 'in_reply_to', 'retweeted_status', 'quoted_status', 'media_tags', 'favorite_count', 'retweet_count', 'bookmark_count', 'quote_count', 'reply_count', 'views_count', 'favorited', 'retweeted', 'bookmarked', 'url', 'text_raw', 'is_retweet', 'hashtags_list', 'text_clean_llm', 'lost_hashtags_count']
🔎 Traitement de tout le fichier: 5 lignes


,id,created_at,full_text,media,screen_name,name,profile_image_url,user_id,in_reply_to,retweeted_status,...,views_count,favorited,retweeted,bookmarked,url,text_raw,is_retweet,hashtags_list,text_clean_llm,lost_hashtags_count
0,1343458257915031553,2020-12-28 08:26:23 +01:00,"💩 à @free parce-que Débit Très instable, … \n\...",[],m_annuel,M Annuel,https://abs.twimg.com/sticky/default_profile_i...,1104790986801250304,NaN,NaN,...,NaN,False,False,False,https://twitter.com/m_annuel/status/1343458257...,"💩 à @free parce-que Débit Très instable, … F...",False,"['#fing', '#internet', '#Free']","💩 à @free parce-que débit très instable, … fre...",0
1,1480465391424053249,2022-01-10 10:03:49 +01:00,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,"[{""type"":""video"",""url"":""https://t.co/c1e4svvex...",GroupeIliad,Groupe iliad,https://pbs.twimg.com/profile_images/127365768...,3157961111,NaN,NaN,...,NaN,False,False,False,https://twitter.com/GroupeIliad/status/1480465...,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,False,[],🎂 @free célèbre aujourd'hui 10 ans de révoluti...,0
2,1485331097844129798,2022-01-23 20:18:24 +01:00,Et vous ça se passe comment la fibre ?\n@Sosh_...,"[{""type"":""photo"",""url"":""https://t.co/I41UXroh0...",MitchelMcPat,MitchelMcPat,https://pbs.twimg.com/profile_images/177199285...,906220646492983296,NaN,NaN,...,NaN,False,False,False,https://twitter.com/MitchelMcPat/status/148533...,Et vous ça se passe comment la fibre ? @Sosh_f...,False,[],et vous ça se passe comment la fibre ? @sosh_f...,0
3,1694675653205860558,2023-08-24 13:38:55 +02:00,Dans la lutte contre les installations sauvage...,"[{""type"":""photo"",""url"":""https://t.co/nKKA40veR...",jnbarrot,Jean-Noël Barrot,https://pbs.twimg.com/profile_images/191299208...,865482011523166209,NaN,NaN,...,9127.0,False,False,False,https://twitter.com/jnbarrot/status/1694675653...,Dans la lutte contre les installations sauvage...,False,[],dans la lutte contre les installations sauvage...,0
4,1695912123824546156,2023-08-27 23:32:13 +02:00,"Hello @free, @Freebox et @orange je crois que ...","[{""type"":""photo"",""url"":""https://t.co/IreryVRyF...",rgaidot,ɹˈe͡ɪɡɪz,https://pbs.twimg.com/profile_images/156717273...,1245801,NaN,NaN,...,778.0,False,False,False,https://twitter.com/rgaidot/status/16959121238...,"Hello @free, @Freebox et @orange je crois que ...",False,[],"hello @free, @freebox et @orange je crois que ...",0


## Définition du prompt et des exemples (few-shot)

Ici, on prépare :

- Le *prompt système* (contexte global) lu depuis un fichier `prompt_tweets_free.txt`.
- Un template de message utilisateur qui contient le tweet et un exemple de JSON attendu.
- Une liste d'exemples `FEW_SHOTS` (tweet → réponse JSON) pour montrer au modèle le format voulu.
- La variable `BASE_MESSAGES` qui contient :
  - le message système
  - les exemples few-shot

Cette base sera réutilisée pour chaque appel au modèle.


In [5]:
# Lecture du prompt système depuis un fichier texte
with open("prompt_tweets_free.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()

# Template du message utilisateur (on y insérera chaque tweet)
USER_TEMPLATE = """Tweet:
{tweet}

Réponds en JSON (une seule ligne), exemple:
{{"is_claim":1,"topics":["fibre"],"sentiment":"neg","urgence":"haute","incident":"incident_reseau","confidence":0.82}}"""

# Exemples few-shot (tweet -> JSON attendu)
FEW_SHOTS = [ 
    ("rt @free: découvrez la nouvelle chaîne imearth en 4k !",
     '{"is_claim":0,"topics":["tv"],"sentiment":"neu","urgence":"basse","incident":"information","confidence":0.9}'),
    ("@free panne fibre à cergy depuis 7h, impossible de bosser",
     '{"is_claim":1,"topics":["fibre"],"sentiment":"neg","urgence":"haute","incident":"incident_reseau","confidence":0.9}'),
    ("@freebox non mais vous répondez trois jours après... super le service après vente",
     '{"is_claim":1,"topics":["autre"],"sentiment":"neg","urgence":"moyenne","incident":"processus_sav","confidence":0.85}')
]

# Préparation de la base de messages commune : system + few-shot
BASE_MESSAGES = [{"role": "system", "content": SYSTEM_PROMPT}]
for u_text, y_json in FEW_SHOTS:
    BASE_MESSAGES.append({"role": "user", "content": u_text})
    BASE_MESSAGES.append({"role": "assistant", "content": y_json})

print("Prompt système et few-shots initialisés.")


Prompt système et few-shots initialisés.


## Fonctions utilitaires

Nous allons définir trois fonctions importantes :

1. `chat_ollama_batch(model, prompts)`  
   → envoie les prompts au modèle Ollama (un par un), avec gestion des erreurs et retries.

2. `_norm_incident(x)`  
   → normalise la variable `incident` pour la ramener à un petit set de valeurs autorisées
   (ex. "réseau", "sav", "support" → catégories standard).

3. `parse_json_line(s)`  
   → essaie de parser proprement la réponse du LLM en JSON,
   même si la réponse est entourée de ```json ... ``` ou contient du texte en plus.


In [6]:
def chat_ollama_batch(model: str, prompts: list[str]) -> list[str]:
    """
    Appelle Ollama (API chat) de manière séquentielle avec retries/backoff.
    - `prompts` : liste de textes (un par tweet)
    - Retour : liste de réponses brutes (strings), une par prompt
    """
    out: list[str] = []

    for p in prompts:
        # messages = system + few-shots + message utilisateur
        messages = BASE_MESSAGES + [{"role": "user", "content": p}]
        payload = {
            "model": model,
            "messages": messages,
            "stream": False,
            "options": {"temperature": 0.1}
        }

        last_err = None
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                r = SESSION.post(API_URL, json=payload, timeout=HTTP_TIMEOUT)
                r.raise_for_status()
                data = r.json()
                content = data.get("message", {}).get("content", "").strip()
                if content:
                    out.append(content)
                    break
                else:
                    last_err = "empty_content"
                    raise RuntimeError("Empty content from model")
            except Exception as e:
                last_err = str(e)
                # si erreur, on attend un peu avant de retenter
                if attempt < MAX_RETRIES:
                    time.sleep(BACKOFFS[min(attempt-1, len(BACKOFFS)-1)])
                else:
                    # si toutes les tentatives échouent, on renvoie un JSON d'erreur minimal
                    safe_err = last_err.replace('"', "'") if last_err else "unknown_error"
                    out.append(
                        '{"is_claim":0,"topics":["autre"],"sentiment":"neu","urgence":"basse","incident":"autre","confidence":0.0,"_error":"%s"}'
                        % safe_err
                    )
    return out


def _norm_incident(x: str) -> str:
    """
    Normalise la valeur 'incident' vers un set de valeurs autorisées,
    en gérant quelques alias courants.
    """
    allowed = {"facturation", "incident_reseau", "livraison", "information", "processus_sav", "autre"}
    s = str(x).strip().lower().replace(" ", "_")

    aliases = {
        "reseau": "incident_reseau",
        "réseau": "incident_reseau",
        "incident": "incident_reseau",
        "sav": "processus_sav",
        "service_apres_vente": "processus_sav",
        "service_après_vente": "processus_sav",
        "support": "processus_sav",
        "facture": "facturation",
        "info": "information",
        "livraisons": "livraison"
    }

    s = aliases.get(s, s)
    return s if s in allowed else "autre"


def parse_json_line(s: str) -> dict:
    """
    Parse robuste de la ligne JSON renvoyée par le LLM.
    - Gère les blocs ```json ... ```
    - Essaie d'extraire la partie entre { et }
    - Renvoie un dict par défaut si tout échoue
    """
    s = str(s).strip()

    # Gestion des blocs ```json ... ```
    if s.startswith("```"):
        lines = s.splitlines()
        if len(lines) >= 3 and lines[0].startswith("```") and lines[-1].startswith("```"):
            s = "\n".join(lines[1:-1]).strip()
        else:
            # fallback : on enlève les backticks et un éventuel 'json'
            s = s.strip("`").strip()
            if s.lower().startswith("json"):
                s = s[4:].strip()

    # Première tentative : parse direct
    try:
        return orjson.loads(s)
    except Exception:
        pass

    # Deuxième tentative : on isole le premier { et le dernier }
    try:
        start = s.find("{")
        end = s.rfind("}")
        if start != -1 and end != -1 and end > start:
            return orjson.loads(s[start:end+1])
    except Exception:
        pass

    # Fallback : objet par défaut
    return {
        "is_claim": 0,
        "topics": ["autre"],
        "sentiment": "neu",
        "urgence": "basse",
        "incident": "autre",
        "confidence": 0.0,
        "_parse_error": True
    }

print("Fonctions utilitaires prêtes.")


Fonctions utilitaires prêtes.


## Préparation des textes à envoyer au modèle

On va maintenant :

- Extraire la colonne de texte `TEXT_COL` du DataFrame `df`.
- La convertir en liste de chaînes de caractères (`texts`).
- Vérifier rapidement le contenu (quelques premiers tweets).


In [7]:
# Extraction de la colonne texte :
texts = df[TEXT_COL].astype(str).tolist()

print(f"Nombre de tweets à traiter : {len(texts)}")
print("Exemples de tweets :")
for t in texts[:5]:
    print("-", t)


Nombre de tweets à traiter : 5
Exemples de tweets :
- 💩 à @free parce-que débit très instable, … free en france: vitesse, performances, pannes et commentaires des utilisateurs 👉 #fing #internet #free via @fingapp @outagedetect
- 🎂 @free célèbre aujourd'hui 10 ans de révolution mobile ! notre cp 👉
- et vous ça se passe comment la fibre ? @sosh_fr @covageidf_ouest c’est la misère vraiment jpp
- dans la lutte contre les installations sauvages et les infrastructures dégradées, la bonne formation des techniciens qui déploient la fibre est essentielle. j'étais ce matin aux côtés d'idriss, formateur au centre de formation technique @free à argenteuil, dans le val-d'oise.
- hello @free, @freebox et @orange je crois que nous allons avoir prochainement un gros problème de fibre / pboe


## Application du modèle par batch

Dans cette étape, on :

1. Parcourt la liste des tweets par batch de taille `BATCH_SIZE`.
2. Pour chaque batch :
   - On génère les prompts à partir du `USER_TEMPLATE`.
   - On appelle `chat_ollama_batch` pour obtenir les réponses du LLM.
   - On parse chaque réponse JSON avec `parse_json_line`.
   - On normalise les champs (`topics`, `sentiment`, `urgence`, `incident`, `is_claim`, `confidence`).
3. On stocke toutes les prédictions dans une liste `results` (une entrée par tweet).


In [8]:
results: list[dict] = []

for i in range(0, len(texts), BATCH_SIZE):
    batch = texts[i:i + BATCH_SIZE]
    
    # Création des prompts pour chaque tweet du batch
    prompts = [USER_TEMPLATE.format(tweet=t) for t in batch]
    
    # Appel au modèle LLM via Ollama
    responses = chat_ollama_batch(MODEL, prompts)
    
    # Traitement de chaque réponse brute
    for raw in responses:
        obj = parse_json_line(raw)

        # ---- Normalisation des topics ----
        topics_val = obj.get("topics", [])
        if isinstance(topics_val, str):
            topics = [topics_val.lower()]
        else:
            try:
                topics = [str(t).lower() for t in topics_val]
            except TypeError:
                topics = [str(topics_val).lower()]
        obj["topics"] = topics

        # ---- Normalisation des autres champs ----
        obj["sentiment"] = str(obj.get("sentiment", "neu")).lower()
        obj["urgence"]   = str(obj.get("urgence", "basse")).lower()
        obj["incident"]  = _norm_incident(obj.get("incident", "autre"))

        # Cast de is_claim en int
        try:
            obj["is_claim"] = int(obj.get("is_claim", 0))
        except Exception:
            obj["is_claim"] = 0

        # Cast de confidence en float
        try:
            obj["confidence"] = float(obj.get("confidence", 0.0))
        except Exception:
            obj["confidence"] = 0.0

        results.append(obj)
    
    print(f"Traités: {min(i + BATCH_SIZE, len(texts))}/{len(texts)}")

print("Classification terminée.")


Traités: 5/5
Classification terminée.


## Fusion des résultats avec les données d'origine et sauvegarde

Maintenant que nous avons :

- `df` : le DataFrame original (tweets + colonnes d'origine)
- `results` : une liste de dictionnaires contenant les prédictions du LLM

Nous allons :

1. Vérifier que le nombre de prédictions == nombre de lignes dans `df`.
2. Transformer `results` en DataFrame `df_pred`.
3. Concaténer `df` et `df_pred` horizontalement.
4. Sauvegarder le résultat final dans `OUTPUT_CSV`.
5. Calculer quelques statistiques simples (ex. taux de réclamations).


In [9]:
# Vérification de cohérence :
if len(results) != len(df):
    raise RuntimeError(
        f"Nombre de résultats ({len(results)}) différent du nombre de lignes ({len(df)})"
    )

# Transformation en DataFrame
df_pred = pd.DataFrame(results)

# Concaténation avec les données originales
out_df = pd.concat([df.reset_index(drop=True), df_pred.reset_index(drop=True)], axis=1)

# Sauvegarde dans un CSV
out_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Fichier de résultats écrit dans : {OUTPUT_CSV}")

# Affichage de quelques stats
if "is_claim" in out_df.columns:
    rate = (out_df["is_claim"] == 1).mean()
    print(f"Réclamations détectées: {rate:.1%}")

out_df.head()


✅ Fichier de résultats écrit dans : C:\Users\HP\Desktop\projet\data\resultat_classification_echantillion_5_tweets.csv
Réclamations détectées: 60.0%


,id,created_at,full_text,media,screen_name,name,profile_image_url,user_id,in_reply_to,retweeted_status,...,is_retweet,hashtags_list,text_clean_llm,lost_hashtags_count,is_claim,topics,sentiment,urgence,incident,confidence
0,1343458257915031553,2020-12-28 08:26:23 +01:00,"💩 à @free parce-que Débit Très instable, … \n\...",[],m_annuel,M Annuel,https://abs.twimg.com/sticky/default_profile_i...,1104790986801250304,NaN,NaN,...,False,"['#fing', '#internet', '#Free']","💩 à @free parce-que débit très instable, … fre...",0,1,[wifi],neg,moyenne,incident_reseau,0.95
1,1480465391424053249,2022-01-10 10:03:49 +01:00,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,"[{""type"":""video"",""url"":""https://t.co/c1e4svvex...",GroupeIliad,Groupe iliad,https://pbs.twimg.com/profile_images/127365768...,3157961111,NaN,NaN,...,False,[],🎂 @free célèbre aujourd'hui 10 ans de révoluti...,0,0,[autre],pos,basse,information,1.00
2,1485331097844129798,2022-01-23 20:18:24 +01:00,Et vous ça se passe comment la fibre ?\n@Sosh_...,"[{""type"":""photo"",""url"":""https://t.co/I41UXroh0...",MitchelMcPat,MitchelMcPat,https://pbs.twimg.com/profile_images/177199285...,906220646492983296,NaN,NaN,...,False,[],et vous ça se passe comment la fibre ? @sosh_f...,0,1,[fibre],neg,haute,incident_reseau,0.95
3,1694675653205860558,2023-08-24 13:38:55 +02:00,Dans la lutte contre les installations sauvage...,"[{""type"":""photo"",""url"":""https://t.co/nKKA40veR...",jnbarrot,Jean-Noël Barrot,https://pbs.twimg.com/profile_images/191299208...,865482011523166209,NaN,NaN,...,False,[],dans la lutte contre les installations sauvage...,0,0,[autre],neu,basse,information,0.95
4,1695912123824546156,2023-08-27 23:32:13 +02:00,"Hello @free, @Freebox et @orange je crois que ...","[{""type"":""photo"",""url"":""https://t.co/IreryVRyF...",rgaidot,ɹˈe͡ɪɡɪz,https://pbs.twimg.com/profile_images/156717273...,1245801,NaN,NaN,...,False,[],"hello @free, @freebox et @orange je crois que ...",0,1,[fibre],neg,haute,incident_reseau,0.90
